# LiDAR Uncertainty at SOAP and SJER NEON sites

## The SOAP site
![Soaproot panorama](https://www.neonscience.org/sites/default/files/styles/he/public/image-content-images/Soaproot_pano.jpg?h=38da9059&itok=XfdBWdLh)
Image credit: National Ecological Observation Network, available at https://www.neonscience.org/field-sites/soap

In [1]:
import os
import pathlib

import earthpy as et
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterstats as rs
import xarray as xr 
import rioxarray as rxr 
import seaborn as sns

et.data.get_data('spatial-vector-lidar')

home_dir = os.path.join(
    pathlib.Path.home(),
    'earth-analytics',
    'data',
    'spatial-vector-lidar')
os.chdir(home_dir)

NEONDataLoader object:
  - takes:
    - name of the dataset
    - id column name
    - dictionary of formatting to apply to file paths
    - id modifier
  - has:
    - name of the dataset
    - id column name
    - paths
    - LiDAR GeoDataFrame
    - insitu DataFrame
    - merged GeoDataFrame
  - does:
    - plot the data
    - (caching)
    - (save the plot to a file)

In [63]:
class NEONDataLoader:
    """Parent class to load NEON height data"""

    base_dir_tmpl = os.path.join(
        'california',
        'neon-{site_name_low}-site')
    insitu_path_tmpl = os.path.join(
        '{base_dir}',
        '2013',
        'insitu',
        'veg{separator}structure',
        'D17_2013_{site_name_up}_vegStr.csv')
    chm_path_tmpl = os.path.join(
        '{base_dir}',
        '2013',
        'lidar',
        '{site_name_up}_lidarCHM.tif')
    plots_path_tmpl = os.path.join(
        '{base_dir}',
        'vector_data',
        '{site_name_up}{plot}_centroids.shp')
    
    site_name = NotImplemented
    id_col_name = NotImplemented
    formatting_dict = NotImplemented
    id_modifier = None

    def __init__(self):
        self.formatting_dict['site_name_low'] = self.site_name.lower()
        self.formatting_dict['site_name_up'] = self.site_name.upper()
        self.formatting_dict['base_dir'] = (
            self.base_dir_tmpl.format(**self.formatting_dict))

        self.insitu_path = self.insitu_path_tmpl.format(**self.formatting_dict)
        self.chm_path = self.chm_path_tmpl.format(**self.formatting_dict)
        self.plots_path = self.plots_path_tmpl.format(**self.formatting_dict)

        self._insitu_height_stats = None
        self._lidar_chm_stats = None
        self._height_stats = None

    @property
    def lidar_chm_stats(self):
        """
        Calculate max and mean tree height from LiDAR
        """
        if self._lidar_chm_stats is None:
            plots_gdf = gpd.read_file(self.plots_path)
            plots_gdf.geometry = plots_gdf.geometry.buffer(20)

            # Calculate the zonal stats
            chm_stats = rs.zonal_stats(
                plots_gdf, self.chm_path,
                stats=['mean', 'max'], nodata=0,
                geojson_out=True, copy_properties=True)
            self._lidar_chm_stats = gpd.GeoDataFrame.from_features(chm_stats)
            self._lidar_chm_stats.rename(
                columns={'max': 'lidar_max', 'mean': 'lidar_mean'},
                inplace=True)
            if not self.id_modifier is None:
                self._lidar_chm_stats[self.id_col_name] = (
                    self._lidar_chm_stats[self.id_col_name]
                    .apply(self.id_modifier))
        return self._lidar_chm_stats
    
    @property
    def insitu_height_stats(self):
        """
        Calculate insitu tree height data max and mean.
        """
        if self._insitu_height_stats is None:
            self._insitu_height_stats = (
                pd.read_csv(self.insitu_path)
                .groupby('plotid')
                .stemheight
                .agg(['max', 'mean'])
                .rename(columns={'max': 'insitu_max', 
                                 'mean': 'insitu_mean'}))
        return self._insitu_height_stats
    
    @property
    def height_stats(self):
        """
        Calculate LiDAR and insitu height stats.
        """
        if self._height_stats is None:
            self._height_stats = (
                self.lidar_chm_stats
                .merge(self.insitu_height_stats, 
                    right_index=True, 
                    left_on=self.id_col_name))
        return self._height_stats

In [64]:
class SJERDataLoader(NEONDataLoader):

    site_name = 'SJER'
    id_col_name = 'Plot_ID'
    formatting_dict = {
        'separator': '_', 
        'plot': '_plot'}

sjer_data_loader = SJERDataLoader()
sjer_data_loader.height_stats.head()

,geometry,Plot_ID,Point,easting,northing,plot_type,lidar_max,lidar_mean,insitu_max,insitu_mean
0,"POLYGON ((255872.376 4111567.818, 255872.280 4...",SJER1068,center,255852.376,4111567.818,trees,19.049999,11.544347,19.3,3.866667
1,"POLYGON ((257426.967 4111298.971, 257426.871 4...",SJER112,center,257406.967,4111298.971,trees,24.019999,10.369277,23.9,8.221429
2,"POLYGON ((256858.760 4110819.876, 256858.664 4...",SJER116,center,256838.760,4110819.876,grass,16.070000,7.518398,16.0,8.218750
3,"POLYGON ((256196.947 4108752.026, 256196.851 4...",SJER117,center,256176.947,4108752.026,trees,11.059999,7.675346,11.0,6.512500
4,"POLYGON ((255988.372 4110476.079, 255988.276 4...",SJER120,center,255968.372,4110476.079,grass,5.740000,4.591177,8.8,7.600000


In [69]:
class SOAPDataLoader(NEONDataLoader):

    site_name = 'SOAP'
    id_col_name = 'ID'
    formatting_dict = {
        'separator': '-', 
        'plot': ''}
    
    def id_modifier(self, id):
        return 'SOAP' + str(id)

soap_data_loader = SOAPDataLoader()
soap_data_loader.height_stats.head()

,geometry,ID,OBJECTID,Protocols,plotDimens,plotType,subtype,lidar_max,lidar_mean,insitu_max,insitu_mean
1,"POLYGON ((297065.197 4100713.028, 297065.101 4...",SOAP43,2,"beetles, soils, coarse-downed wood, leaf area ...",40.0,distributed,base,54.099998,19.804080,51.1,5.047000
2,"POLYGON ((299825.197 4101013.028, 299825.101 4...",SOAP63,3,"beetles, soils, coarse-downed wood, leaf area ...",40.0,distributed,base,32.480000,16.989834,33.0,9.232787
4,"POLYGON ((298715.197 4100833.028, 298715.101 4...",SOAP95,5,"beetles, soils, coarse-downed wood, leaf area ...",40.0,distributed,base,33.869999,17.412048,28.1,6.050943
5,"POLYGON ((297065.197 4100083.028, 297065.101 4...",SOAP139,6,"beetles, soils, coarse-downed wood, leaf area ...",40.0,distributed,base,49.919998,19.417840,120.0,4.745902
6,"POLYGON ((299885.197 4100413.028, 299885.101 4...",SOAP143,7,"beetles, soils, coarse-downed wood, leaf area ...",40.0,distributed,base,27.340000,10.454653,19.7,2.530702
